In [1]:
import pandas as pd
import numpy as np


In [3]:
!python -V

Python 3.9.12


In [24]:
# !pip install pyarrow
!pip install -U scikit-learn

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import Ridge


from sklearn.metrics import mean_squared_error

In [26]:
pd.__version__

'1.4.2'

In [3]:
df= pd.read_parquet('../notebooks/data/yellow_tripdata_2023-01.parquet')

In [4]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df['duration_min'] = round(df[df.tpep_pickup_datetime.dt.month ==1].duration.dt.total_seconds() /60 , 2)

In [66]:
df[df['duration_min'] >= 60]

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee,duration,duration_min
59,2,2023-01-01 00:22:39,2023-01-01 01:30:45,1.0,16.02,1.0,N,132,265,2,...,1.0,0.5,0.00,0.00,1.0,64.85,0.0,1.25,0 days 01:08:06,68.10
383,2,2023-01-01 00:36:07,2023-01-02 00:17:13,1.0,1.20,1.0,N,234,234,1,...,1.0,0.5,0.00,0.00,1.0,16.40,2.5,0.00,0 days 23:41:06,1421.10
533,2,2023-01-01 00:21:42,2023-01-01 23:18:28,2.0,3.05,1.0,N,48,140,1,...,1.0,0.5,3.72,0.00,1.0,28.52,2.5,0.00,0 days 22:56:46,1376.77
546,2,2023-01-01 00:11:01,2023-01-01 23:37:57,1.0,1.68,1.0,N,138,129,1,...,6.0,0.5,3.89,0.00,1.0,23.34,0.0,1.25,0 days 23:26:56,1406.93
760,1,2023-01-01 00:38:25,2023-01-01 01:38:39,1.0,13.70,1.0,N,161,200,1,...,3.5,0.5,15.80,3.00,1.0,94.70,2.5,0.00,0 days 01:00:14,60.23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3066030,1,2023-01-31 16:49:45,2023-01-31 17:56:16,NaN,0.00,NaN,None,163,132,0,...,2.5,0.5,19.03,6.55,1.0,98.08,NaN,NaN,0 days 01:06:31,66.52
3066068,2,2023-01-31 17:38:06,2023-01-31 18:43:18,NaN,21.90,NaN,None,143,132,0,...,0.0,0.5,10.00,6.55,1.0,97.55,NaN,NaN,0 days 01:05:12,65.20
3066081,2,2023-01-31 17:44:41,2023-01-31 18:48:49,NaN,18.57,NaN,None,237,132,0,...,0.0,0.5,16.20,0.00,1.0,97.20,NaN,NaN,0 days 01:04:08,64.13
3066159,2,2023-01-31 17:30:32,2023-01-31 18:37:19,NaN,14.84,NaN,None,141,132,0,...,0.0,0.5,8.10,0.00,1.0,89.10,NaN,NaN,0 days 01:06:47,66.78


In [7]:
import sklearn

In [8]:
sklearn.__version__

'1.6.1'

In [4]:
df.columns

Index(['VendorID', 'tpep_pickup_datetime', 'tpep_dropoff_datetime',
       'passenger_count', 'trip_distance', 'RatecodeID', 'store_and_fwd_flag',
       'PULocationID', 'DOLocationID', 'payment_type', 'fare_amount', 'extra',
       'mta_tax', 'tip_amount', 'tolls_amount', 'improvement_surcharge',
       'total_amount', 'congestion_surcharge', 'airport_fee'],
      dtype='object')

# Q1


In [18]:
len(df.columns)

19

# Q2

### My Solution

In [5]:
df.tpep_dropoff_datetime = pd.to_datetime(df.tpep_dropoff_datetime)
df.tpep_pickup_datetime = pd.to_datetime(df.tpep_pickup_datetime)

In [6]:
df["duration"] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: round(td.total_seconds() / 60,2))

In [7]:
df.duration.dtypes

dtype('float64')

In [8]:
round(df[df.tpep_pickup_datetime.dt.month ==1]['duration_min'].std(),2)

42.59

# Q3

In [9]:
df.duration_min.describe(percentiles=[0.95, 0.98, 0.99])

count    3.066718e+06
mean     1.566837e+01
std      4.258564e+01
min     -2.920000e+01
50%      1.152000e+01
95%      3.647000e+01
98%      4.873000e+01
99%      5.725000e+01
max      1.002918e+04
Name: duration_min, dtype: float64

In [10]:
df[(df.duration_min>=1) & (df.duration_min<=60)].duration_min.describe(percentiles=[0.95, 0.98, 0.99])

count    3.009136e+06
mean     1.420488e+01
std      9.939403e+00
min      1.000000e+00
50%      1.155000e+01
95%      3.477000e+01
98%      4.450000e+01
99%      5.027000e+01
max      6.000000e+01
Name: duration_min, dtype: float64

In [11]:
((df.duration_min>=1) & (df.duration_min<=60)).mean()

0.9812082173860021

# Q4

In [12]:
df = df[(df.duration_min >= 1) & (df.duration_min <= 60)]

In [13]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']

In [14]:
df[categorical]=df[categorical].astype(str)

In [15]:
train_dicts = df[categorical + numerical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)



In [16]:
len(dv.feature_names_)

516

# Q5

In [17]:
target= 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)


LinearRegression()

In [18]:
y_pred = lr.predict(X_train)

In [ ]:
sns.distplot(y_pred, label='prediction')
sns.distplot(y_train, label='actual')



In [19]:
mean_squared_error(y_train, y_pred, squared=False)

TypeError: got an unexpected keyword argument 'squared'